In [1]:
!pip install xgboost

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
time: 949 ms (started: 2021-12-20 04:37:34 +00:00)


In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

train_df = pd.read_csv('task1_2021.training.csv')
test_df = pd.read_csv('task1_2021.test.csv')
train_y = train_df['target']
train_x = train_df.drop(['target'], axis=1)

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_df)

mse = make_scorer(mean_squared_error, squared=False)

time: 335 ms (started: 2021-12-20 04:40:43 +00:00)


In [2]:
from sklearn.svm import SVR 


param_dist = {
    'C':range(20,200,1),
    'kernel':['poly', 'rbf', 'sigmoid', 'precomputed'],
    'tol':np.linspace(1e-5,1e-3,100)
}

svr_model = SVR()

grid = RandomizedSearchCV(svr_model, param_dist, cv=3, scoring=mse, n_iter=1000, verbose=3, n_jobs=-1)
grid.fit(train_x, train_y)
# print(cross_val_score(svr_model, train_x, train_y,cv=5,scoring=mse).mean())

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


ValueError: X should be a square kernel matrix

time: 9min 45s (started: 2021-12-20 04:41:37 +00:00)


In [ ]:
print(grid.best_score_)
svr_model = grid.best_estimator_

In [ ]:
import xgboost as xgb
# read in data
# dtrain = xgb.DMatrix(data=train_x, label=train_y)
# dtest = xgb.DMatrix(data=test_x)
# specify parameters via map
# param_dist = {'n_estimators':1000,'max_depth':10, 'learning_rate':0.01, 'objective':'reg:squarederror', 'booster':'dart', 'n_jobs':-1}
param_dist = {
    'n_estimators':range(100,10000,1),
    'max_depth':range(10,100,1),
    'learning_rate':np.linspace(0.001,0.01,10),
    # 'subsample':np.linspace(0.,0.9,20),
    # 'colsample_bytree':np.linspace(0.5,0.98,10),
    # 'min_child_weight':range(1,9,1),
    'booster':['dart'],
    'objective':['reg:squarederror'],
    'verbosity':[1],
    'n_jobs':[-1]
    
}

bstr_model = xgb.XGBRegressor()
grid = RandomizedSearchCV(bstr_model, param_dist, cv=3, scoring=mse, n_iter=10, verbose=3, n_jobs=-1)
grid.fit(train_x, train_y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [ ]:
print(grid.best_score_)
bstr_model = grid.best_estimator_
# make prediction
# preds = bst.predict(dtrain)
# print(mean_squared_error(train_y, preds, squared=False))
# print(cross_val_score(bstr_model, train_x, train_y, cv=5, scoring=mse))

In [ ]:
from pandas import RangeIndex

svr_pred = svr_model.predict(test_x)
bst_pred = bstr_model.predict(dtest)

output = bst_pred
pd.Series(output, index=RangeIndex(1, 2001), name='Predicted').to_csv('ans7.csv',index_label='id')

In [ ]:
output = (svr_pred+bst_pred)/2
pd.Series(output, index=RangeIndex(1, 2001), name='Predicted').to_csv('ans8.csv',index_label='id')